In [1]:
import pandas as pd
import requests, sys
import json
import os
import urllib.request
from Bio import SeqIO, pairwise2, AlignIO
import re
import html
from collections import defaultdict
import subprocess
from Bio.PDB.DSSP import make_dssp_dict
import mapping_uniprot_pdb

/Users/charlottecrauwels/anaconda3/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
#Download file with classification info
filename_listGPCRdb = "../data/250220_Classification_GPCRdb.xlsx"
listGPCRdb_df = pd.read_excel(filename_listGPCRdb)

#chimeric design info
filename_chimeric_designs = "../data/previous_designs.xlsx"
chimeric_design_df = pd.read_excel(filename_chimeric_designs)

#alignment all natural GPCRs
MSA = "../data/MSA_all_mammalian.fasta"

#chimeras
chimeric_entry_data = "../data/all_designs.fasta"
chimeras_record_dict = SeqIO.index(chimeric_entry_data, "fasta")

# file with the representative experimental 3D structures (lowest resolution independent of activation state)
# file generate with the notebook "reference_structure.ipynb" updated Feb 2025
representative_structures_json = json.load(open("../data/250225_representative_structures_exp_pdbID_uniprotID.json"))


In [3]:
#upload data
of_interest=['OPRM_MOUSE_OPRK_HUMAN_1',"NTR1_HUMAN_OPRK_HUMAN_1","SSR2_HUMAN_OPRK_HUMAN_1","HRH2_HUMAN_OPRK_HUMAN_1","ADA1A_HUMAN_OPRK_HUMAN_1","CXCR3_HUMAN_OPRK_HUMAN_1",
             "ADA1A_HUMAN_OPRK_HUMAN_2","OPSD_BOVIN_ACM3_HUMAN_1"]

entry_data = "../data/all_designs.fasta"
entry_uniprotID_seq = {}
for record in SeqIO.parse(entry_data,"fasta"):
    if record.id in of_interest:
        entry_uniprotID_seq[record.id]=str(record.seq)

print(len(entry_uniprotID_seq))
print(entry_uniprotID_seq)

8
{'OPSD_BOVIN_ACM3_HUMAN_1': 'MNGTEGPNFYVPFSNKTGVVRSPFEAPQYYLAEPWQFSMLAAYMFLLIMLGFPINFLTLYVFKVNKQLKTVLNYILLNLAVADLFMVFGGFTTTLYTSLHGYFVFGPTGCNLEGFFATLGGEIALWSLVVLAIERYVVVTRPLTYRAKRTTKRAIMGVAFTWVMALACAAPPLVGWSRYIPEGMQCSCGIDYYTPHEETNNESFVIYMFVVHFIIPLIVIFFCYGRIYKETEKRTKELAGLQASGTEAETENFVHPTGSSRSCSSYELQQQSMKRSNRRKYGRCHFWFTTKSWKPSSEQMDQDHSSSDSWNNNDAAASLENSASSDEEDIGSETRAIYSIVLKLPGHSTILNSTKLPSSDNLQVPEEELGMVDLERKADKLQAQKSVDDGGSFPKSFSKLPIQLESAVDTAKTSDVNSSVGKSTATLPLSFKEATLAKRFALKTRSQITKRKRMSLVKEKKAARMVIIMVIAFLICWLPYAGVAFYIFTHQGSDFGPIFMTIPAFFAKTSAVYNPVIYIMMNKQFRTTFKMLLLCQCDKKKRRKQQYQQRQSVIFHKRAPEQALTETSQVAPA', 'OPRM_MOUSE_OPRK_HUMAN_1': 'MDSSAGPGNISDCSDPLAPASCSPAPGSWLNLSHVDGNQSDPCGPNRTGLGGSHSLCPQTGSPSMVTAITIMALYSIVCVVGLFGNFLVMYVIVRYTKMKTATNIYIFNLALADALATSTLPFQSVNYLMGTWPFGNILCKIVISIDYYNMFTSIFTLCTMSVDRYIAVCHPVKALDFRTPRNAKIVNVCNWILSSAIGLPVMFMATTKYRQGSIDCTLTFSHPTWYWENLLKICVFIFAFIMPVLIITVCYGLMILRLKSVRLLSGSREKDRNLRRITRMVLVVVAVFIVCWTPIHIYVIIKALITIPETTFQTVSWHFCIALGYTNSCLNPVLYAFLDENFKRCFREFCIPTSSTIEQQNSAR

In [4]:
def calculate_sq_atom_distance(i, j):
    """Squared euclidean distance between two 3d points"""
    return (i[0] - j[0]) * (i[0] - j[0]) + \
            (i[1] - j[1]) * (i[1] - j[1]) + \
            (i[2] - j[2]) * (i[2] - j[2])

def identify_gaps(pdb_file, chain_pdb, offset, end): #code modified from pdb_gap.py file from pdbtools Copyright 2018 João Pedro Rodrigues
    fhandle = open(pdb_file, 'r')
    centroid = ' CA '  # respect spacing. 'CA  ' != ' CA '
    distance_threshold = 4.0 * 4.0
    prev_at = (None, None, None, None, (None, None, None))
    model = 0
    n_gaps = 0
    gap = []
    for line in fhandle:

        if line.startswith('MODEL'):
            model = int(line[10:14])

        elif line.startswith('ATOM'):
            atom_name = line[12:16]
            if atom_name != centroid:
                continue

            resn = line[17:20]
            resi = int(line[22:26])
            chain = line[21]
            x = float(line[30:38])
            y = float(line[38:46])
            z = float(line[46:54])

            at_uid = (model, chain, resi, resn, atom_name, (x, y, z))
            if prev_at[0] == at_uid[0] and prev_at[1] == at_uid[1]:
                d = calculate_sq_atom_distance(at_uid[5], prev_at[5])
                if d > distance_threshold:
                    gap.append([prev_at[1],prev_at[2],at_uid[1],at_uid[2]])
                    # sys.stdout.write(fmt_GAPd.format(prev_at, at_uid, d**0.5))
                    n_gaps += 1
                elif prev_at[2] + 1 != at_uid[2]:
                    # sys.stdout.write(fmt_GAPs.format(prev_at, at_uid))
                    gap.append([prev_at[1],prev_at[2],at_uid[1],at_uid[2]])
                    n_gaps += 1

            prev_at = at_uid

    gaps_cleaned = []
    start = offset
    for section in gap:
        if section[0] == chain_pdb and section[2] == chain_pdb:
            stop = section[1]
            if start < 1000 and stop < 1000:
                gaps_cleaned.append([start,stop])
            start = section[3]
    gaps_cleaned.append([start,end])
    return gaps_cleaned

In [ ]:
def merge_duplicates(dicts):
    # Step 1: Group dictionaries by the 'start' key
    grouped = defaultdict(list)
    for d in dicts:
        grouped[d['start']].append(d)
    
    result = []
    conflicts = []

    # Step 2: Process each group
    for start, items in grouped.items():
        if len(items) > 1:
            # Check if all 'type' values are the same
            types = set(d['type'] for d in items)
            if len(types) == 1:
                # Merge 'reference' values
                merged_references = ""
                for d in items:
                    merged_references += d['description']
                # Create a new dictionary with merged references
                new_dict = items[0].copy()
                new_dict['description'] = merged_references[:-1]
                new_dict['reference'] = "https://www.ebi.ac.uk/pdbe/pisa/"
                result.append(new_dict)
            else:
                # Print dictionaries with different 'type' values
                for d in items:
                    conflicts.append(d)
        else:
            result.append(items[0])
    
    return result, conflicts

def retrieve_interacting_residues_PDB(pdb_id, chain_pdb,mapping_uniprot_PDB_dict,pdb_file_path):
    mapping_PDB_uniprot = {v: k for k, v in mapping_uniprot_PDB_dict.items()} #gives position of a aligned res in unaligned seq

    # retrieve the interacting residues in the PDBs from PISA, need to make sure it doesn't take into accound the interactions between 2 sym GPCRs
    # interacting residues is defined by a bsa > 0
    #https://github.com/PDBe-KB/pdbe-pisa-json/blob/main/PISA-APIs.ipynb

    interacting_residues_list = []
    binders_chain= []
    # for pdb_id, chain_pdb,uniprot_pdb_start,pdb_start in zip(pdb_ids,chain_pdbs,uniprot_pdb_starts,pdb_starts):
    # try: #when its just 1 chain or 1 chain and a ligand PISA doesn't work
        
    if "a"=="a":
        
        response = requests.get(f"https://www.ebi.ac.uk/pdbe/api/pisa/assembly/{pdb_id.lower()}/1")
        interface_count = response.json()[pdb_id.lower()]["assembly"]["interface_count"]
        for i in range(1,interface_count+1):
            interacting_residues = []
            response_single_interface = requests.get(f"https://www.ebi.ac.uk/pdbe/api/pisa/interface/{pdb_id.lower()}/1/{i}/")
            data = response_single_interface.json()
            if "/" in chain_pdb:
                chain_pdb = chain_pdb.split("/")
            for j in range(len(data["molecules"])):
                if isinstance(chain_pdb,str):
                    if data["molecules"][j]["chain_id"]==chain_pdb:
                        for bsa,position in zip(data["molecules"][j]["buried_surface_areas"],data["molecules"][j]['residue_seq_ids']):
                            if bsa > 0.0:
                                try:
                                    interacting_residues.append(mapping_PDB_uniprot[int(position)])
                                except:
                                    continue
                        if j == 0: #there is supposed to be only 2 molecules, the GPCR and the interacting molecule
                            chain_interacting_molecule = data["molecules"][1]["chain_id"]
                        else: 
                            chain_interacting_molecule = data["molecules"][0]["chain_id"]
                        binders_chain.append(extract_name_binders(chain_interacting_molecule,pdb_file_path))
                        interacting_residues_list.append(list(set(interacting_residues)))

                elif isinstance(chain_pdb,list):
                    if chain_pdb[0] in data["molecules"][j]["chain_id"] and chain_pdb[1] in data["molecules"][j+1]["chain_id"]:
                        break
                    else:
                        if chain_pdb[0] in data["molecules"][j]["chain_id"] or chain_pdb[1] in data["molecules"][j]["chain_id"]:
                            for bsa,position in zip(data["molecules"][j]["buried_surface_areas"],data["molecules"][j]['residue_seq_ids']):
                                if bsa >0.0:
                                    try:
                                        interacting_residues.append(mapping_PDB_uniprot[int(position)])
                                    except:
                                        continue
                        if j == 0: #there is supposed to be only 2 molecules, the GPCR and the interacting molecule
                            chain_interacting_molecule = data["molecules"][1]["chain_id"]
                        else: 
                            chain_interacting_molecule = data["molecules"][0]["chain_id"]
                        binders_chain.append(extract_name_binders(chain_interacting_molecule,pdb_file_path))
                        interacting_residues_list.append(list(set(interacting_residues)))

        return interacting_residues_list,binders_chain
    # except:
    #     return [],""


def extract_name_binders(chain_of_interest,pdb_file_path):

    molecule_name = None
    current_molecule = ""
    reading_molecule = False
    found_chain = False
    
    # Open and read the PDB file
    with open(pdb_file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        if line.startswith("COMPND"):
            # Start reading the molecule name if "MOLECULE" is in the line
            if "MOLECULE" in line:
                reading_molecule = True
                current_molecule = line.split(":")[1].strip().rstrip(";")  # Extract initial part of the molecule name
            # If the molecule name is being read and it continues on the next line
            elif reading_molecule and "CHAIN" not in line:
                pattern = r"\d+\s+(.+)"
                match = re.search(pattern, line)
                current_molecule += " "+match.group(1).strip().rstrip(";")
            # Once we reach the chain of interest
            if bool(re.search(rf"CHAIN:\s*(?:[^,]*,\s*)*{chain_of_interest}\b", line)):
                found_chain = True
            # If molecule and chain have been found, stop reading
            if found_chain and current_molecule and ";" in line:
                molecule_name = current_molecule
                break
    return molecule_name
 

In [6]:
from Bio import PDB

def extract_resolution_and_method(file_path):
    """
    Extracts resolution and experimental method from a PDB or mmCIF file.

    Parameters:
    file_path (str): Path to the structure file (.pdb or .cif).

    Returns:
    dict: A dictionary containing the resolution and experimental method.
    """
    method_abb = {"ELECTRON MICROSCOPY":"EM","X-RAY DIFFRACTION":"X-RAY","SOLUTION NMR":"NMR", "SOLID-STATE NMR":"NMR",
                  "ELECTRON CRYSTALLOGRAPHY":"EM"}
    
    if file_path.endswith(".cif"):
        parser = PDB.MMCIFParser(QUIET=True)
    elif file_path.endswith(".pdb"):
        parser = PDB.PDBParser(QUIET=True)
    else:
        raise ValueError("Unsupported file format. Please provide a .pdb or .cif file.")

    structure = parser.get_structure("structure", file_path)

    # Extracting metadata from the structure header
    header = structure.header

    resolution = str(header.get("resolution", "Not available"))+"Å"
    experimental_method = method_abb[header.get("structure_method", "Not available").upper()]
    if experimental_method == "NMR":
        resolution = ""

    return resolution,experimental_method

In [ ]:
def retrieve_pdb_dsbonds_chimera_info(structures_pdb,uniprot_id,sequence):
    structures = []
    ds_bonds = []
    pdbs = []
    chains = []
    uniprot_pdb_starts = []
    pdb_starts = []
    binders_list= []
    interacting_residues_list = []

    for pdb_info in structures_pdb:
        pdb_id = pdb_info["PDB id"]
        chain_pdb = pdb_info["chain"]
        state = pdb_info["state"]
        if not os.path.exists('../data/tmp/'):
            os.mkdir('../data/tmp/')
        try:
            mmcif=False
            pdb_file_path = f'../data/tmp/{pdb_id}.pdb'
            urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb_id}.pdb', pdb_file_path)
        except:
            mmcif=True
            # pdb_file_path = f'../data/tmp/{pdb_id}.cif'
            # urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb_id}.cif', pdb_file_path)
        
        if not mmcif:
            #get uniprot to pdb mapping
            folder_mapping_json = "../examples/3Dstructures/uniprot_pdb_mapping/"
            mapping_file = folder_mapping_json+pdb_id+".json"
            if not os.path.exists(folder_mapping_json+pdb_id+".json"):
                mapping_uniprot_pdb_dict= mapping_uniprot_pdb.map_PDB_uniprot(pdb_id,pdb_file_path,chain_pdb,uniprot_id,sequence,folder_mapping_json, type_gpcr = "chimera")
            else:
                mapping_uniprot_pdb_dict = json.load(open(mapping_file))
                mapping_uniprot_pdb_dict = {int(k): v for k, v in mapping_uniprot_pdb_dict.items()} #keys are strings


            # #this is needed to find the interactions within the pdb file
            pdbs.append(pdb_id)

            #get offset and gaps in structure
            resolution,method =  extract_resolution_and_method(pdb_file_path)
            if mmcif:
                url = f"https://files.rcsb.org/download/{pdb_id}.cif"
            else:
                url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
            
            structures.append({"offset":  0, "gaps": [],"value":pdb_id,"chain": chain_pdb, "state":state, "mapping": f"file:///examples/3Dstructures/uniprot_pdb_mapping/{pdb_id}.json", "resolution": resolution, "method": method, "url":url, "reference":f"https://www.rcsb.org/structure/{pdb_id.upper()}", "date":""})
            
            #find interacting residues at ligand binding site and G protein binding site
            interacting_residues,binders = retrieve_interacting_residues_PDB(pdb_id,chain_pdb,mapping_uniprot_pdb_dict,pdb_file_path)
            if len(interacting_residues)>0:
                interacting_residues_list.append(interacting_residues)
                binders_list.append(binders)


            #remove pdb file
            os.remove(pdb_file_path)
            
    return structures, interacting_residues_list, binders_list, pdbs

In [8]:
def run_dssp(uniprotID, type ="natural"):

    dssp_folder = "../examples/3Dstructures/dssp/"
    dssp_filename = dssp_folder+uniprotID+".dssp"
    remove_tmp = False
    mapping_uniprot_pdb_numbering = None

    if not os.path.exists(dssp_filename):
        try: 
            pdb_representative = representative_structures_json[uniprotID]["pdb_id"]
            gpcr_chain = representative_structures_json[uniprotID]["gpcr_chain"]
            print("Problem, GPCR has exp structure and has no DSSP file", pdb_representative, gpcr_chain)
            #add chimera exp structure
        except:
            #AF
            #check if gpcrdb has updated model
            if type == "natural":
                if os.path.exists(f"../examples/3Dstructures/AF_gpcrdb_2024/{uniprotID}.pdb"):
                    input_file = f"../examples/3Dstructures/AF_gpcrdb_2024/{uniprotID}.pdb"
                    gpcr_chain = "A"
                else:
                    #AlphaFold2 DB
                    input_file = f'../examples/3Dstructures/tmp/{uniprotID}.pdb'
                    urllib.request.urlretrieve(f"https://alphafold.ebi.ac.uk/files/AF-{uniprotID}-F1-model_v4.pdb", input_file)
                    gpcr_chain = "A"
                    remove_tmp = True
            else:
                if os.path.exists(f"../examples/3Dstructures/AF_chimera_2024/{uniprotID}.pdb"):
                    input_file = f"../examples/3Dstructures/AF_chimera_2024/{uniprotID}.pdb"
                    gpcr_chain = "A"
            
            subprocess.run(
                    ["mkdssp", input_file, dssp_filename])
                
            if remove_tmp:
                os.remove(input_file)
        return dssp_filename, gpcr_chain, mapping_uniprot_pdb_numbering

    else:
        try: 
            pdb_representative = representative_structures_json[uniprotID]["pdb_id"]
            gpcr_chain = representative_structures_json[uniprotID]["gpcr_chain"]
            mapping_uniprot_pdb_numbering = json.load(open("../examples/3Dstructures/uniprot_pdb_mapping/"+pdb_representative+".json"))
            mapping_uniprot_pdb_numbering = {int(k): v for k, v in mapping_uniprot_pdb_numbering.items()} #keys are strings
        except: #AF model
            gpcr_chain = "A"
        return dssp_filename, gpcr_chain, mapping_uniprot_pdb_numbering

def find_closest(target,dict_interest):
    if target in dict_interest:
        min_start = dict_interest[target]
    else:
        closest_key = min(dict_interest.keys(), key=lambda x: abs(x - target))
        min_start = dict_interest[closest_key]
    return min_start
    
def remap_min_max_limits_prot_interest(min_max_limits,aligned_seq_interest):
    translate_seq_MSA = map_seq_MSA(aligned_seq_interest) #gives position of a unaligned res in msa
    translate_MSA_seq = {v: k for k, v in translate_seq_MSA.items()} #gives position of a aligned res in unaligned seq
    min_max_limits_translated = {}
    for TMname, limits in min_max_limits.items():
        min_start = find_closest(limits[0][0],translate_MSA_seq)
        max_start = find_closest(limits[0][1],translate_MSA_seq)
        min_end = find_closest(limits[1][0],translate_MSA_seq)
        max_end = find_closest(limits[1][1],translate_MSA_seq)
        min_max_limits_translated[TMname]=[[min_start,max_start],[min_end,max_end]]
    return min_max_limits_translated


def refine_TM_regions(min_max_limits_translated,TM_regions_prot):

    refined_TMs = {}
    
    # Step 1: Ensure each TM aligns with MSA-defined limits
    for tm_label, (dssp_start, dssp_end) in TM_regions_prot:
        if tm_label in min_max_limits_translated:
            (min_start, max_start), (min_end, max_end) = min_max_limits_translated[tm_label]

            # Step 2: Adjust start and end points based on MSA limits
            adjusted_start = max(min_start, min(dssp_start, max_start))  # Keep within min-max range
            adjusted_end = min(max_end, max(dssp_end, min_end))  # Keep within min-max range

            # Ensure the TM region has at least 10 residues
            if adjusted_end - adjusted_start + 1 < 10:
                # If too short, expand towards the closest allowed limit
                if adjusted_start > min_start:
                    adjusted_start = max(min_start, adjusted_start - (10 - (adjusted_end - adjusted_start + 1)))
                if adjusted_end < max_end:
                    adjusted_end = min(max_end, adjusted_end + (10 - (adjusted_end - adjusted_start + 1)))

            refined_TMs[tm_label] = (adjusted_start, adjusted_end)

    # Step 3: Handle cases where DSSP predicts fewer than 7 TMs
    if len(refined_TMs) < 7:
        missing_TMs = [tm for tm in min_max_limits_translated.keys() if tm not in refined_TMs]
        for tm in missing_TMs:
            (min_start, max_start), (min_end, max_end) = min_max_limits_translated[tm]
            refined_TMs[tm] = (min_start, max_end)  # Assign entire range if missing

    # Step 4: Handle cases where DSSP predicts too many TMs
    if len(refined_TMs) > 7:
        # Merge small or overlapping helices
        tm_keys = sorted(refined_TMs.keys(), key=lambda x: int(x[2:]))  # Sort TM labels (TM1, TM2, ...)
        merged_TMs = {}
        prev_tm = None

        for tm in tm_keys:
            if prev_tm is None:
                merged_TMs[tm] = refined_TMs[tm]
            else:
                prev_start, prev_end = merged_TMs[prev_tm]
                curr_start, curr_end = refined_TMs[tm]

                # Merge if overlap or short segment
                if curr_start - prev_end < 5 or (curr_end - curr_start + 1 < 10):
                    merged_TMs[prev_tm] = (prev_start, curr_end)  # Extend previous TM
                else:
                    merged_TMs[tm] = refined_TMs[tm]

            prev_tm = tm
        
        refined_TMs = merged_TMs

    # Step 5: Ensure exactly 7 TM regions
    if len(refined_TMs) < 7:
        missing_TMs = [tm for tm in min_max_limits_translated.keys() if tm not in refined_TMs]
        for tm in missing_TMs[: 7 - len(refined_TMs)]:  # Fill in missing TMs up to 7
            (min_start, max_start), (min_end, max_end) = min_max_limits_translated[tm]
            refined_TMs[tm] = (min_start, max_end)

    return refined_TMs

def map_min_max_limitsMSA_chimera(min_max_limits_translated_parent,aligned_seq_interest_ref,seq_chimera):

    unaligned_ref = aligned_seq_interest_ref.replace("-","")

    alignments_global = pairwise2.align.globalms(
        unaligned_ref, seq_chimera, match=2, mismatch=-1,
        open=-10, extend=-2,
        one_alignment_only=True
    )

    aligned_ref_MSAchimera, aligned_seq_interest_chimera = alignments_global[0].seqA, alignments_global[0].seqB

    #map min max limits TMs parent to MSA with chimera
    min_max_limits_MSAchimera = {}
    translate_seq_MSA = map_seq_MSA(aligned_ref_MSAchimera) #gives position of a unaligned res in msa
    for TMname, limits in min_max_limits_translated_parent.items():
        min_start = translate_seq_MSA[limits[0][0]]
        max_start = translate_seq_MSA[limits[0][1]]
        min_end = translate_seq_MSA[limits[1][0]]
        max_end = translate_seq_MSA[limits[1][1]]
        min_max_limits_MSAchimera[TMname]=[[min_start,max_start],[min_end,max_end]]
    return min_max_limits_MSAchimera,aligned_seq_interest_chimera

def compute_dssp_TM_regions(uniprotID,MSA,type_gpcr = "natural",ref_id=None, seq_chimera = None):

    record_dict = SeqIO.index(MSA, "fasta")
    if type_gpcr == "natural":
        aligned_seq_interest = str(record_dict[uniprotID].seq)
        length_prot = len(aligned_seq_interest.replace("-",""))
    else:
        length_prot = len(seq_chimera)
        aligned_seq_interest_ref = str(record_dict[ref_id].seq)
    
    dssp_filename, gpcr_chain, mapping_uniprot_pdb_numbering  = run_dssp(uniprotID,type_gpcr) #run dssp if needed on AF model else retrieve dssp file
    dssp_tup = make_dssp_dict(dssp_filename)
    dssp_dic = dssp_tup[0]

    # Extract DSSP codes along with their actual residue positions
    dssp_positions = []
    for key, value in dssp_dic.items():
        chain, res_info = key  # Extract chain and residue details
        if chain == gpcr_chain:  # Filter only the chain of interest
            res_id = res_info[1]  # Actual residue position in the protein
            dssp_code = value[1]  # DSSP secondary structure code
            dssp_positions.append((res_id, dssp_code))

    # Replace '-' with 'X' in DSSP codes
    dssp_positions = [(res_id, code.replace('-', 'X')) for res_id, code in dssp_positions]

    # Define conserved secondary structure elements (Helix structures)
    conserved_2structure_dssp = {"H", "I", "G"}

    # Filter for helix positions
    helix_positions = [res_id for res_id, code in dssp_positions if code in conserved_2structure_dssp]

    if mapping_uniprot_pdb_numbering != None:
        helix_positions_renumbered = []
        mapping_pdb_uniprot_numbering = {v: k for k, v in mapping_uniprot_pdb_numbering.items()}
        for pos in helix_positions:
            try:
                helix_positions_renumbered.append(mapping_pdb_uniprot_numbering[int(pos)])
            except: #not part of GPCR
                continue
        helix_positions = helix_positions_renumbered

    TM_regions = []
    #find start and end anchor points
    start = helix_positions[0]
    stop = None
    counter = 1
    min_tm_length = 10 #a TM are typically at least 10 residues long otherwise they are considered as small helices connecting the TMs
    for i in range(1, len(helix_positions)):
        if helix_positions[i] - helix_positions[i - 1] > 1:
            if helix_positions[i-1] - start >= min_tm_length :
                stop = helix_positions[i-1]
            else:
                start = helix_positions[i]
        if stop != None:
            # if stop-start+1 > 3: #we need at least 3 consecutive columns with enough conserved secondary structure elements before considering it as an anchor region
            TM_regions.append(("TM"+str(counter),(start,stop)))
            counter +=1
            stop = None
            start = helix_positions[i]
    TM_regions.append(("TM"+str(counter),(start,helix_positions[-1])))
    
    #adapt TM regions based on 50%-80% limits
    min_max_limits = {'TM1':[[559,563],[588,588]],
                        'TM2':[[606,606],[636,637]],
                        'TM3':[[654,657],[690,690]],
                        'TM4':[[717,717],[741,742]],
                        'TM5':[[913,916],[946,948]],
                        'TM6':[[1169,1174],[1203,1204]],
                        'TM7':[[1246,1248],[1270,1271]],} #based on partial dssp MSA with mapping dssp exp structures representative
    
    if type_gpcr == "natural":
        min_max_limits_translated_interest = remap_min_max_limits_prot_interest(min_max_limits,aligned_seq_interest)
        refined_TM_regions = refine_TM_regions(min_max_limits_translated_interest,TM_regions)
    else:

        min_max_limits_translated_parent = remap_min_max_limits_prot_interest(min_max_limits,aligned_seq_interest_ref)
        min_max_limits_MSAchimera_parent,aligned_seq_interest_chimera = map_min_max_limitsMSA_chimera(min_max_limits_translated_parent,aligned_seq_interest_ref,seq_chimera)
        min_max_limits_translated_interest = remap_min_max_limits_prot_interest(min_max_limits_MSAchimera_parent,aligned_seq_interest_chimera)
        refined_TM_regions = refine_TM_regions(min_max_limits_translated_interest, TM_regions)
    
    structured_regions = [
        ("Nterm", 1, refined_TM_regions["TM1"][0] - 1),
        ("TM1", refined_TM_regions["TM1"][0], refined_TM_regions["TM1"][1]),
        ("ICL1", refined_TM_regions["TM1"][1] + 1, refined_TM_regions["TM2"][0] - 1),
        ("TM2", refined_TM_regions["TM2"][0],refined_TM_regions["TM2"][1]),
        ("ECL1", refined_TM_regions["TM2"][1] + 1, refined_TM_regions["TM3"][0] - 1),
        ("TM3",  refined_TM_regions["TM3"][0], refined_TM_regions["TM3"][1]),
        ("ICL2",  refined_TM_regions["TM3"][1] + 1,  refined_TM_regions["TM4"][0] - 1),
        ("TM4",  refined_TM_regions["TM4"][0],refined_TM_regions["TM4"][1]),
        ("ECL2", refined_TM_regions["TM4"][1] + 1, refined_TM_regions["TM5"][0] - 1),
        ("TM5", refined_TM_regions["TM5"][0],refined_TM_regions["TM5"][1]),
        ("ICL3", refined_TM_regions["TM5"][1] + 1, refined_TM_regions["TM6"][0] - 1),
        ("TM6", refined_TM_regions["TM6"][0],refined_TM_regions["TM6"][1]),
        ("ECL3", refined_TM_regions["TM6"][1] + 1, refined_TM_regions["TM7"][0] - 1),
        ("TM7", refined_TM_regions["TM7"][0],refined_TM_regions["TM7"][1]),
        ("H8&Cterm", refined_TM_regions["TM7"][1] + 1, length_prot)
    ]

    regions = []
    for name, start, end in structured_regions:
        regions.append({
            "name": name,
            "start": start,
            "end": end,
            "reference": "DSSP"
        })

    return regions

In [9]:
def cutting_pts_2_ss_region(dict_regions,all_regions=None):

    translated_regions = []
    if isinstance(dict_regions,dict):
        for region in dict_regions.keys():
            lower_lim = dict_regions[region][0]
            upper_lim = dict_regions[region][1]
            for ss_region in all_regions:
                if lower_lim >= ss_region["start"] and lower_lim <= ss_region["end"]:
                    ss_lower_lim = ss_region["name"]
                if upper_lim >= ss_region["start"] and upper_lim <= ss_region["end"]:
                    ss_upper_lim = ss_region["name"]
            translated_regions.append(str(lower_lim)+"-"+str(upper_lim)+f" ({ss_lower_lim}-{ss_upper_lim})")

    elif isinstance(dict_regions,list):
        if all_regions:
            for section in dict_regions:
                info = []
                for pos in section:
                    for ss_region in all_regions:
                        if int(pos) >= ss_region["start"] and int(pos) <= ss_region["end"]:
                            ss_pos = ss_region["name"]
                            break
                    info.append(pos)
                    info.append(ss_pos)
                translated_regions.append(str(info[0])+"-"+str(info[2])+f" ({info[1]}-{info[3]})")
        else:
            for region in dict_regions:
                lower_lim = region[0]
                upper_lim = region[1]
                translated_regions.append(str(lower_lim)+"-"+str(upper_lim))
    return translated_regions


def sections_coloring_chimera(info_specific_chimera,allregions):
    coloring = {"Nterm":{0:"",1:"",2:"",3:"",4:""},
                "TM1":{0:"",1:"",2:"",3:"",4:""},
                "TM1-Loop-TM2":{0:"",1:"",2:"",3:"",4:""},
                "TM2":{0:"",1:"",2:"",3:"",4:""},
                "TM2-Loop-TM3":{0:"",1:"",2:"",3:"",4:""},
                "TM3":{0:"",1:"",2:"",3:"",4:""},
                "TM3-Loop-TM4":{0:"",1:"",2:"",3:"",4:""},
                "TM4":{0:"",1:"",2:"",3:"",4:""},
                "TM4-Loop-TM5":{0:"",1:"",2:"",3:"",4:""},
                "TM5":{0:"",1:"",2:"",3:"",4:""},
                "TM5-Loop-TM6":{0:"",1:"",2:"",3:"",4:""},
                "TM6":{0:"",1:"",2:"",3:"",4:""},
                "TM6-Loop-TM7":{0:"",1:"",2:"",3:"",4:""},
                "TM7":{0:"",1:"",2:"",3:"",4:""},
                "H8&Cterm":{0:"",1:"",2:"",3:"",4:""}}

    cutting_pts_with_ss = info_specific_chimera["cutting_point_chimera"]

    for parent,region in enumerate(cutting_pts_with_ss):

        positions=region.split(" ")[0]
        name=region.split(" ")[1][1:-1]
        info = []
        for i in range(1):
            position = int(positions.split("-")[i])
            name_region= name.split("-")[i]
            lower_lim = [d for d in allregions if d.get("name") == name_region][0]["start"]
            upper_lim = [d for d in allregions if d.get("name") == name_region][0]["end"]
            a_fifth = round((upper_lim-lower_lim)/5)

            if position < (lower_lim + a_fifth):
                idx = 0
            elif position < (lower_lim + 2*a_fifth):
                idx = 1
            elif position < (lower_lim + 3*a_fifth):
                idx = 2
            elif position < (lower_lim + 4*a_fifth):
                idx = 3
            else:
                idx = 4

            #make names match
            name_loops = {"ICL1":"TM1-Loop-TM2","ECL1":"TM2-Loop-TM3",
                            "ICL2":"TM3-Loop-TM4","ECL2":"TM4-Loop-TM5",
                            "ICL3":"TM5-Loop-TM6","ECL3":"TM6-Loop-TM7"}
            if "IC" in name_region or "EC" in name_region:
                name_region = name_loops[name_region]

            if (parent % 2)==0:
                parent_name = "EC"
            else:
                parent_name = "IC"

            if len(coloring[name_region][idx]) != 0:
                coloring[name_region][idx+1]=parent_name
            else:
                coloring[name_region][idx]=parent_name
            
    content = "EC"
    for region_coloring, idx_coloring in coloring.items():
            for idx in idx_coloring.keys():
                if len(idx_coloring[idx]) == 0:
                    idx_coloring[idx] = content
                else:
                    content = idx_coloring[idx]
               
    return coloring

def retrieve_involvement_natural_chimeric_design(uniprot_id,all_regions):

    involvement = []

    for parent_column_id in ['Reference_id','Target_id']:

        #find rows that have uniprot as ref id or target id
        designs_parent = chimeric_design_df[chimeric_design_df[parent_column_id] == uniprot_id]

        #Info from rows
        names_chimeras = designs_parent['Chimera_name'].tolist()
        ids_chimeras = designs_parent['Chimera_name_ids'].tolist()
        regions_chimera = designs_parent['Chimera_parts'].tolist()
        name_target_chimeras = designs_parent['Target_name'].tolist()
        id_target_chimeras = designs_parent['Target_id'].tolist()

        name_ref_chimeras = designs_parent['Reference_name'].tolist()
        id_ref_chimeras = designs_parent['Reference_id'].tolist()

        regions_ref_chimeras = designs_parent['Reference_cutting_points'].tolist()
        regions_target_chimeras = designs_parent['Target_cutting_points'].tolist()

        expression = designs_parent['Expression binary'].tolist()
        fct = designs_parent['Function binary'].tolist()

        application = designs_parent['Application'].tolist()
        type_chimera = designs_parent['Chimera Type (1/2/3)'].tolist()
        Gprot = designs_parent['G-protein'].tolist()
        Ligand =designs_parent['Ligand'].tolist()
        structures = designs_parent['3D structure PDB'].tolist()
        biblio = designs_parent['DOI'].tolist()

        for i,(name, id) in enumerate(zip(names_chimeras,ids_chimeras)):
            sequence_chimera = str(chimeras_record_dict[name].seq)
            all_regions = compute_dssp_TM_regions(id,MSA,type_gpcr = "chimera",ref_id=id_ref_chimeras[i], seq_chimera = sequence_chimera)
            cutting_pt_chimera = cutting_pts_2_ss_region(eval(regions_chimera[i]),all_regions) 

            all_regions_ref = compute_dssp_TM_regions(id_ref_chimeras[i],MSA,type_gpcr = "natural")
            cutting_pt_ref = cutting_pts_2_ss_region(eval(regions_ref_chimeras[i]),all_regions_ref)

            all_regions_target = compute_dssp_TM_regions(id_target_chimeras[i],MSA,type_gpcr = "natural")
            cutting_pt_target = cutting_pts_2_ss_region(eval(regions_target_chimeras[i]),all_regions_target)

            pharma_name_ref = html.unescape(get_pharma_name(id_ref_chimeras[i],name_ref_chimeras[i]))
            pharma_name_target = html.unescape(get_pharma_name(id_target_chimeras[i], name_target_chimeras[i]))

            pharma_name_ref_ = pharma_name_ref
            pharma_name_target_ = pharma_name_target
            if "receptor" in pharma_name_ref.lower():
                pharma_name_ref_ = pharma_name_ref.replace(" receptor","")
            if "receptor" in pharma_name_target.lower():
                pharma_name_target_ = pharma_name_target.replace(" receptor","")
            pharma_name = pharma_name_ref_ + " " + pharma_name_target_ + " receptor"
            if "adrenoceptor" in pharma_name:
                pharma_name = pharma_name.replace(" receptor","")

            if isinstance(structures[i],str):
                pdb = structures[i]
            else:
                pdb = ""

            if isinstance(Gprot[i],str):
                gprot=Gprot[i]
            else:
                gprot=""

            if isinstance(Ligand[i],str):
                ligand=Ligand[i]
            else:
                ligand=""

            chimera={
            "name":name,
            "name_pharma":pharma_name,
            "id":ids_chimeras[i],
            "ref": name_ref_chimeras[i],
            "ref_pharma_name": pharma_name_ref,
            "target": name_target_chimeras[i],
            "target_pharma_name":pharma_name_target,
            "cutting_point_chimera": cutting_pt_chimera,
            "cutting_point_ref": cutting_pt_ref,
            "cutting_point_target": cutting_pt_target,
            "expression_function": fct[i],
            "type": type_chimera[i],
            "GprotLigand": gprot+" "+ligand,
            "application": application[i]+" "+pdb,
            "reference": biblio[i]
            }
            involvement.append(chimera)

    return involvement


In [10]:
def retrieve_3D_structure_info(info_3Dstructure):
    structures_pdb = []
    for pdb_info in info_3Dstructure.split(","):
        pdb_id = pdb_info.split('(')[0]
        chain = pdb_info.split('(')[1].split(' ')[0]
        state = pdb_info.split('(')[1].split(' ')[1][:-1]
        pdb_dict={"PDB id":pdb_id,"chain":chain,"state":state}
        structures_pdb.append(pdb_dict)
    return structures_pdb

def split_string(input_string):
    match = re.match(r"([a-zA-Z]?)(-?)(\d*)(-?)([a-zA-Z]?)", input_string)
    if match:
        return list(filter(None, match.groups()))
    return []

def retrieve_mutations_vs_parents_info(mutations,biblio):
    mutations_list = []
    if len(mutations)>=4:
        for mut in mutations.split(","):
            if len(mut)>3:
                split_mut = split_string(mut)
                position = split_mut[1]
                parent_res = split_mut[0]
                chimera_res = split_mut[2]
                mut_dict = {"start":int(position),"end":int(position),"original residue":parent_res, "alternative residue":chimera_res,"reference":biblio}
                mutations_list.append(mut_dict)
        return mutations_list
    return TypeError

def retrieve_chimera_design_info(abb_name):

    all_info = chimeric_design_df[chimeric_design_df["Chimera_name"] == abb_name]

    uniprot_id = all_info["Chimera_name_ids"].values[0]
    given_name = all_info["Given name"].values[0]
    cutting_points = eval(all_info["Chimera_parts"].values[0])
    ref_parent_name = all_info["Reference_name"].values[0]
    ref_parent_id = all_info["Reference_id"].values[0]
    target_parent_name = all_info["Target_name"].values[0]
    target_parent_id = all_info["Target_id"].values[0]
    Gprot = all_info['G-protein'].values[0]

    if isinstance(Gprot,str):
        gprot=Gprot
    else:
        gprot=""  
    
    biblio = all_info['DOI'].values[0]
    try:
        structures_pdb =  retrieve_3D_structure_info(all_info["3D structure PDB"].values[0])
    except:
        structures_pdb = []
    try:
        mutations_vs_parents = retrieve_mutations_vs_parents_info(all_info["Mutations"].values[0],biblio)
    except:
        mutations_vs_parents = []

    return uniprot_id,given_name,cutting_points,ref_parent_name,ref_parent_id,target_parent_name,target_parent_id,structures_pdb,mutations_vs_parents,gprot,biblio

In [11]:
def familly_subclass_parents(ref_parent_name,ref_parent_id,target_parent_name,target_parent_id):
    #find classfication based on human classification in GPCRdb
    #find human ortholog
    family_chimera = []
    subclass_ligand_chimera = []
    subclass_phylo_chimera = []

    for abbreviated_name,uniprotID in zip([ref_parent_name,target_parent_name],[ref_parent_id,target_parent_id]):
        try:
            if listGPCRdb_df[listGPCRdb_df['Uniprot ID'] == uniprotID]["Phylogenetically-based"].values[0] == "A-other":
                if not abbreviated_name.endswith("HUMAN"):
                    abbreviated_name_human = (abbreviated_name.split('_')[0]+"_"+"HUMAN").lower()
                    uniprot_id_human = listGPCRdb_df[listGPCRdb_df['Name'] == abbreviated_name_human]['Uniprot ID'].values[0] 
            else:
                uniprot_id_human = uniprotID


            family = listGPCRdb_df[listGPCRdb_df['Uniprot ID'] == uniprot_id_human]["Subclass"].values[0].rstrip() #need to change this to interpro API for those not on GPCRdb


            if "Class A" in family:
                family = family.replace("Class A ","")
                family = family[0].upper()+family[1:]
            if "receptors" in family:
                family = family.replace("receptors","").rstrip()
            elif "receptor" in family:
                family = family.replace("receptor","").rstrip()
            subclass_ligand = listGPCRdb_df[listGPCRdb_df['Uniprot ID'] == uniprot_id_human]["Ligand-based"].values[0].rstrip() #need to change so it works for all mammals (put same as what we have for humans? What with those not on GPCRdb?)

            
            if "receptors" in subclass_ligand:
                subclass_ligand = subclass_ligand.replace("receptors","").rstrip()
            elif "receptor" in subclass_ligand:
                subclass_ligand = subclass_ligand.replace("receptor","").rstrip()
            subclass_phylo = listGPCRdb_df[listGPCRdb_df['Uniprot ID'] == uniprot_id_human]["Phylogenetically-based"].values[0]
            if "A-" in subclass_phylo:
                subclass_phylo = subclass_phylo.split('-')[1].rstrip()
                subclass_phylo = subclass_phylo[0].upper()+subclass_phylo[1:]

        except:
                family = "Olfactory"
                subclass_ligand = "Olfactory"
                subclass_phylo = "Olfactory"

        family_chimera.append(family)
        subclass_ligand_chimera.append(subclass_ligand)
        subclass_phylo_chimera.append(subclass_phylo)
    
    family_chimera_str = " & ".join(family_chimera)
    subclass_ligand_chimera_str = " & ".join(subclass_ligand_chimera)
    subclass_phylo_chimera_str = " & ".join(subclass_phylo_chimera)
    
    return family_chimera_str,subclass_ligand_chimera_str,subclass_phylo_chimera_str

In [12]:
def write_cutting_point_region(cutting_points,ref_parent_name,target_parent_name,biblio):
    cutting_points_regions_parent = []
    for i,region in enumerate(cutting_points):
        if (i%2)==0:
            parent = ref_parent_name
            parent_type = "EC side parent"
        else:
            parent = target_parent_name
            parent_type = "IC side parent"

        region_parent =  {
        "name": f"{parent} ({parent_type})",
        "start": region[0],
        "end": region[1],
        "reference": biblio
        }
        cutting_points_regions_parent.append(region_parent)
    return cutting_points_regions_parent

In [13]:
#convert scientific name UniProt to common name
def parse_species_file(file_path):
    species_dict = {}
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    scientific_name = None
    
    for line in lines:
        if "N=" in line:
            scientific_name = line.split("N=")[1].strip()
        elif "C=" in line and scientific_name:
            common_name = line.split("C=")[1].strip()
            species_dict[scientific_name] = common_name
            scientific_name = None  # Reset for the next entry
    
    return species_dict

def species_uniprot(ref_parent_id,target_parent_id):
    species_parents = []
    for prot_id in [ref_parent_id,target_parent_id]:
        requestURL = f"https://rest.uniprot.org/uniprotkb/{prot_id}.json"
        r = requests.get(requestURL, headers={ "Accept" : "application/json"})
        if not r.ok:
            r.raise_for_status()
            sys.exit()
        uniprot_json = json.loads(r.text)
        species = uniprot_json['organism']['scientificName']
        species_parents.append(species)

    #convert scientific names to common names
    uniprot_names = "../data/UniProt_names_scientific_common.txt"
    species_dict = parse_species_file(uniprot_names)
    species_parents_common_name = []
    for species in species_parents:
        species_parents_common_name.append(species_dict[species])
    return " & ".join(species_parents_common_name)

In [14]:
def retrieve_seq_parents(ref_parent_id,target_parent_id):
    record_dict = SeqIO.index(MSA, "fasta")
    ref_seq = str(record_dict[ref_parent_id].seq).replace("-","")
    target_seq = str(record_dict[target_parent_id].seq).replace("-","")
    return ref_seq,target_seq

In [ ]:
#GPCRdb finds sodium pockets
#As microswitches are well defined in literature we can check ourselves if these well knwon microswitches are present in our gpcrs
#All known microswitches in literature for class A
E_DRY_W = {"positions":["3.49", "3.50", "3.51"],"residues":["ED", "R", "WY"], "name": "E/DRY/W motif (ionic lock switch)"}
CWxP = {"positions":["6.47", "6.48", "6.50"], "residues":["C", "W", "P"], "name": "CWxP motif (transmission toggle switch)"}
NPxxY = {"positions":["7.49", "7.50", "7.53"], "residues":["N","P","Y"], "name": "NPxxY motif (tyr toggle switch)"}
PIF = {"positions": ["5.50", "3.40", "6.44"], "residues":["P","I","F"], "name": "PIF motif"}
hydrophobic_lock = {"positions":["3.43","6.40"], "residues":["LVIM", "LVIM"], "name": "hydrophobic lock"}
# ionic_lock = {"positions":["6.30"], "residues":["DE"], "name": "ionic lock"}
#disulfide bond between TM3 and ECL2 is already identified by Uniprot in the "Disulfide bonds" section
#Sodium binding pocket (allosteric action): middle of the 7TMs. Identified by GPCRdb but are the identified ones all of them???

#the positions are in human readable format (not pyton - starts at 0)
MSA_E_DRY_W = {"positions":[684,685,686],"residues":["ED", "R", "WY"], "name": "E/DRY/W motif (ionic lock switch)"}
MSA_CWxP = {"positions":[1191,1192,1194], "residues":["C", "W", "P"], "name": "CWxP motif (transmission toggle switch)"}
MSA_NPxxY = {"positions":[1265,1266,1269], "residues":["N","P","Y"], "name": "NPxxY motif (tyr toggle switch)"}
MSA_PIF = {"positions": [930,675,1187], "residues":["P","I","F"], "name": "PIF motif"}
MSA_hydrophobic_lock = {"positions":[678,1183], "residues":["LVIM", "LVIM"], "name": "Hydrophobic lock"}
# MSA_ionic_lock = {"positions":[1190], "residues":["DE"], "name": "Ionic lock"}
# MSA_sodium_pocket = {"positions":[620,684], "residues":["D","S"], "name": "Sodium binding pocket"}

TM1x50={"positions":[579],"residues":["N"], "name": "1.50 (BW numbering)"}
TM2x50={"positions":[620],"residues":["D"], "name": "2.50 (BW numbering)"}
TM3x50={"positions":[685],"residues":["R"], "name": "3.50 (BW numbering)"}
TM4x50={"positions":[729],"residues":["W"], "name": "4.50 (BW numbering)"}
TM5x50={"positions":[930],"residues":["P"], "name": "5.50 (BW numbering)"}
TM6x50={"positions":[1194],"residues":["P"], "name": "6.50 (BW numbering)"}
TM7x50={"positions":[1266],"residues":["P"], "name": "7.50 (BW numbering)"}

#equivalence between positions in sequence and in MSA
#dictionary with list of list. In every sublist, 2 elements, 1st is the position in sequence, the 2nd the position in MSA
def map_seq_MSA(sequence_aligned):
    previous = 0
    translate = {}
    sequence_nogaps = sequence_aligned.replace("-","")
    for res in range(len(sequence_nogaps)):
        idx_msa = previous + sequence_aligned[previous:].index(sequence_nogaps[res])
        translate[res+1]=idx_msa+1
        previous = idx_msa + 1
    return translate

#Microswitches/motifs - identify them based on their defined columns in mammalian MSA
def motifs_microswitches_literature(MSA,uniprot_id):

    alignment = AlignIO.read(open(MSA), "fasta")
    len_MSA=alignment.get_alignment_length()
    record_dict = SeqIO.index(MSA, "fasta")
    aligned_seq_interest = str(record_dict[uniprot_id].seq)
    translate_seq_MSA = map_seq_MSA(aligned_seq_interest) #gives position of a unaligned res in msa
    translate_MSA_seq = {v: k for k, v in translate_seq_MSA.items()} #gives position of a aligned res in unaligned seq
    # microswitch_types = [MSA_E_DRY_W, MSA_CWxP, MSA_NPxxY, MSA_PIF, MSA_hydrophobic_lock,
    #                      TM1x50,TM2x50,TM3x50,TM4x50,TM5x50,TM6x50,TM7x50]
    microswitch_types = [MSA_E_DRY_W, MSA_CWxP, MSA_NPxxY, MSA_PIF, MSA_hydrophobic_lock]
    
    microswitches = []
    microswitches_residues = []

    for microswitch_type in microswitch_types:
        are_there = []
        for position, residue in zip(microswitch_type["positions"], microswitch_type["residues"]):
            if aligned_seq_interest[position-1] in residue:
                are_there.append(True)
            else:
                are_there.append(False)
        for i, (position, residue) in enumerate(zip(microswitch_type["positions"], microswitch_type["residues"])):
            microswitch_residue = {}
            
            #take into account the possibility that there is a gap at that position in the MSA
            if position in translate_MSA_seq:
                microswitch_residue["start"] = translate_MSA_seq[position]
                microswitch_residue["end"] = translate_MSA_seq[position]
                residue_motif = aligned_seq_interest[position-1]
            else:
                for next in range(position+1,len_MSA):
                    if next in translate_MSA_seq:
                        microswitch_residue["start"] = translate_MSA_seq[next]
                        microswitch_residue["end"] = translate_MSA_seq[next]
                        residue_motif = aligned_seq_interest[next-1]
                        break

            if not all(are_there) and not are_there[i]:
                if residue_motif == "F" and microswitch_type["name"]=="PIF motif":
                    microswitch_residue["description"] = residue_motif + " instead of "+ residue+ " from " + " (part of " + microswitch_type["name"]+ ")"
                elif residue_motif == "R" and microswitch_type["name"]=="E/DRY/W motif (ionic lock switch)":
                    microswitch_residue["description"] = residue_motif + " instead of "+ residue + " (part of " + microswitch_type["name"]+ ")"
                elif "(BW numbering)" in microswitch_type["name"]:
                    microswitch_residue["description"] = residue_motif+ " instead of "+ residue + " " + microswitch_type["name"]
                else:
                    microswitch_residue["description"] = residue_motif + " instead of "+ residue + " (part of " + microswitch_type["name"]+ ")"
            else:
                if residue_motif == "F" and microswitch_type["name"]=="PIF motif":
                    microswitch_residue["description"] = residue_motif + " part of " + microswitch_type["name"] +" and hydrophobic lock"
                elif residue_motif == "R" and microswitch_type["name"]=="E/DRY/W motif (ionic lock switch)":
                    microswitch_residue["description"] = residue_motif+ " part of " + microswitch_type["name"] +" and ionic lock"
                elif "(BW numbering)" in microswitch_type["name"]:
                    microswitch_residue["description"] = residue_motif+ " " + microswitch_type["name"]
                else:
                    microswitch_residue["description"] = residue_motif+ " part of " + microswitch_type["name"]
            if are_there[i]:
                microswitch_residue["conserved"] = "yes"
            else:
                microswitch_residue["conserved"] = "no"
            microswitch_residue["reference"] = "Based on alignment"
            microswitches_residues.append(microswitch_residue)  

    return microswitches_residues

def find_motifs_parent_chimera(cutting_points_parent,microswitches_to_look_at,cutting_points_chimera,counter):
    microswitches_found = []
    positions_BW_50_chimera = []
    for region_parent in cutting_points_parent:
        region_chimera = cutting_points_chimera[counter]
        start_region_chimera = int(region_chimera.split(' ')[0].split('-')[0])
        stop_region_chimera = int(region_chimera.split(' ')[0].split('-')[1])

        start_region_parent = int(region_parent.split(' ')[0].split('-')[0])
        stop_region_parent = int(region_parent.split(' ')[0].split('-')[1])

        if start_region_parent > start_region_chimera:
            difference = start_region_parent - start_region_chimera
        else:
            difference = start_region_chimera - start_region_parent

        for position in range(start_region_parent,stop_region_parent+1):
            motifs_associated_pos = []
                    
            motifs_associated_pos = [item for item in microswitches_to_look_at if item["start"] == position]
            if len(motifs_associated_pos)>0:
                if start_region_parent > start_region_chimera:
                    position_chimera = position - difference
                else:
                    position_chimera = position + difference
                for motif in motifs_associated_pos:
                    motif_chimera = motif.copy()
                    motif_chimera["start"] = position_chimera
                    motif_chimera["end"] = position_chimera
                    microswitches_found.append(motif_chimera)

                    if "BW numbering" in motif_chimera["description"]: #keep position of .50 residues for BW numbering
                        positions_BW_50_chimera.append(position_chimera)

        counter +=2
    return microswitches_found, positions_BW_50_chimera

def computeBW_numbering_chimera(positions_BW_50_chimera,all_regions):
    length_prot = all_regions[-1]["end"]
    uniprot_BW_mapping =  {i + 1: "" for i in range(length_prot)}
    counter = 1

    for TMname in all_regions:
        if "TM" in TMname["name"]:
            position_50_seq = positions_BW_50_chimera[counter-1]
            numberTM = str(counter)+"."

            uniprot_BW_mapping[position_50_seq]+=(numberTM+str(50))

            start = TMname["start"]
            end = TMname["end"]
            distance = 1
            for residue in range(position_50_seq-1,start-1,-1):
                pos = 50-distance
                uniprot_BW_mapping[residue]+=(numberTM+str(pos))
                distance +=1

            distance = 1
            for residue in range(position_50_seq+1,end+1):
                pos = 50+distance
                uniprot_BW_mapping[residue]+=(numberTM+str(pos))
                distance +=1
            
            counter +=1
            
    for pos, value in uniprot_BW_mapping.items():
        if len(value)==0:
            uniprot_BW_mapping[pos]="N/A"
    return uniprot_BW_mapping

def motifs_microswitches_BWnumbering_chimera(all_regions,ref_parent_id,target_parent_id,info_specific_chimera, MSA):
    #find motifs chimera based on motifs parents. For every region coming from a particular parent, look if the parent has motifs in that region. If yes, chimera also has it.

    cutting_points_chimera = info_specific_chimera["cutting_point_chimera"]
    cutting_points_ref = info_specific_chimera["cutting_point_ref"]
    cutting_points_target = info_specific_chimera["cutting_point_target"]

    microswitches_ref = motifs_microswitches_literature(MSA,ref_parent_id)
    microswitches_target = motifs_microswitches_literature(MSA,target_parent_id)
    microswitches_chimera_ref, positions_BW_50_chimera_ref  = find_motifs_parent_chimera(cutting_points_ref,microswitches_ref,cutting_points_chimera,0)
    microswitches_chimera_target, positions_BW_50_chimera_target = find_motifs_parent_chimera(cutting_points_target,microswitches_target,cutting_points_chimera,1)

    microswitches_residues = microswitches_chimera_ref + microswitches_chimera_target
    positions_BW_50_chimera = positions_BW_50_chimera_ref + positions_BW_50_chimera_target

    uniprot_BW_mapping = computeBW_numbering_chimera(positions_BW_50_chimera,all_regions)
    
    return microswitches_residues, uniprot_BW_mapping


def disulfide_bonds_chimera(ref_parent_id,seq_ref,seq_chimera,allregions):

    disulfide_bonds=[]
    #methodology can be optimized 
    #cannot use MSA as Cys in ECL2 => position not conserved
    requestURL = f"https://rest.uniprot.org/uniprotkb/{ref_parent_id}.json"
    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    uniprot_json = json.loads(r.text)
    ref_ssbonds =[]
    for i in range(len(uniprot_json['features'])):
        if uniprot_json['features'][i]['type'] == 'Disulfide bond':
            ref_ssbonds.append( [uniprot_json['features'][i]['location']['start']['value'],
        uniprot_json['features'][i]['location']['end']['value'] ])
   

    alignments_global = pairwise2.align.globalms(
        seq_ref, seq_chimera, match=2, mismatch=-1,
        open=-10, extend=-2,
        one_alignment_only=True
    )

    aligned_ref, aligned_chimera = alignments_global[0].seqA, alignments_global[0].seqB

    #map min max limits TMs parent to MSA with chimera
    translate_seq_MSA_ref = map_seq_MSA(aligned_ref) #gives position of a unaligned res in msa
    translate_seq_MSA_chimera = map_seq_MSA(aligned_chimera) #gives position of a unaligned res in msa
    translate_MSA_seq_chimera = {v: k for k, v in translate_seq_MSA_chimera.items()}

    #if ss in TM3 ECL2 of chimera than very likely the known conserved ss bond
    for CysCys in ref_ssbonds:
        start_ref = CysCys[0]
        aligned_start_ref = translate_seq_MSA_ref[start_ref]
        start_chimera = translate_MSA_seq_chimera[aligned_start_ref]
        end_ref = CysCys[1]
        aligned_end_ref = translate_seq_MSA_ref[end_ref]
        end_chimera = translate_MSA_seq_chimera[aligned_end_ref]

        #this is quite hacky, to be improved!
        region1 = [d for d in allregions if d["start"] <= start_chimera <= d["end"]][0]["name"]
        if region1 == "TM3":
            region2 = [d for d in allregions if d["start"] <= end_chimera <= d["end"]][0]["name"]
            if region2 == "ECL2":
                disulfide_bonds.append({
                    'start':CysCys[0],
                    'end':CysCys[1],
                    "description":"Disulfide bond",
                    "reference": "Inferred from EC parent.",
                })
    
    return disulfide_bonds

In [16]:
def get_pharma_name(uniprotID,abb_name):
    #GtoP or gpcrdb_name or pharmacological name
    try:
        requestURL = f"https://gpcrdb.org/services/protein/accession/{uniprotID}"

        r = requests.get(requestURL, headers={ "Accept" : "application/json"})
        if not r.ok:
            info_entry = None
        else:
            info_entry = json.loads(r.text)
    except:
        info_entry = None

    if not info_entry is None:
        clean_html_tags = re.compile('<.*?>')
        pharma_name = re.sub(clean_html_tags, '', info_entry["name"])
    else:
        pharma_name = abb_name

    return pharma_name

def retrieve_pharma_name_parents(ref_parent_name,ref_parent_id,target_parent_name,target_parent_id):
    pharma_name_ref = get_pharma_name(ref_parent_id,ref_parent_name)
    pharma_name_target = get_pharma_name(target_parent_id,target_parent_name)

    if "receptor" in pharma_name_ref.lower():
        pharma_name_ref = pharma_name_ref.replace(" receptor","")
    if "receptor" in pharma_name_target.lower():
        pharma_name_target = pharma_name_target.replace(" receptor","")
    
    pharma_name = pharma_name_ref + " " + pharma_name_target + " receptor"

    if "adrenoceptor" in pharma_name:
        pharma_name = pharma_name.replace(" receptor","")

    return pharma_name_ref,pharma_name_target,pharma_name

In [17]:
def retrieve_predicted_models(structures,uniprotID):

    #AlphaFold2
    af_model_path = f"file:///examples/3Dstructures/AF_chimera_2024/{uniprotID}.pdb"
    structures.append({"value":f"AlphaFold2","chain": "A", "state":"Undetermined", "offset":  0, "gaps": [], "resolution": "", "method": "Predicted", "url": af_model_path, "reference":"AlphaFold2","date":"2024"})
    
    #AlphaFold multistate. Don't have a AF ms for every GPCR (only humans). Need to check if file exist:
    af_ms_active = f"../examples/3Dstructures/AFms_2023/Active/{uniprotID}.pdb"
    af_ms_inactive = f"../examples/3Dstructures/AFms_2023/Inactive/{uniprotID}.pdb"
    esmf = f"../examples/3Dstructures/ESMF_chimera_2023/{uniprotID}.pdb"
    if os.path.exists(af_ms_active):
        af_ms_active = f"file:///examples/3Dstructures/AFms_2023/Active/{uniprotID}.pdb"
        structures.append({"value":f"AlphaFold2-Multistate Active","chain": "A",  "state":"Active", "offset":  0, "gaps": [], "resolution": "", "method": "Predicted", "url": af_ms_active, "reference":"AlphaFold multistate", "date":"2023"})
    if os.path.exists(af_ms_inactive):
        af_ms_inactive = f"file:///examples/3Dstructures/AFms_2023/Inactive/{uniprotID}.pdb"
        structures.append({"value":f"AlphaFold2-Multistate Inactive","chain": "A",  "state":"Inactive", "offset":  0, "gaps": [], "resolution": "", "method": "Predicted", "url": af_ms_inactive, "reference":"AlphaFold multistate", "date":"2023"})
    if os.path.exists(esmf):
        esmf = f"file:///examples/3Dstructures/ESMF_chimera_2023/{uniprotID}.pdb"
        structures.append({"value":f"ESMFold","chain": "A", "state":"Undetermined", "offset":  0, "gaps": [], "resolution": "", "method": "Predicted", "url": esmf, "reference":"ESMFold", "date":"2023"})

    return structures

In [18]:
def gather_interacting_residues(interacting_residues_list,binders_list,pdbs):
    # Reprocessing the data with the updated binders list
    interactions = {}

    for i, residue_groups in enumerate(interacting_residues_list):
        binders = binders_list[i]
        pdb_id = pdbs[i]

        for j,residue_group in enumerate(residue_groups):
            for residue in residue_group:
                if residue not in interactions:
                    interactions[residue] = [[binders[j],pdb_id]]
                else:
                    interactions[residue].append([binders[j],pdb_id])

    contacts_list =[]
    for residue,info in interactions.items():
        pdbs = []
        binders = []
        for data in info:
            if not data[0] is None:
                binders.append(data[0])
                pdbs.append(data[1])
        binders=list(set(binders))
        if len(binders)>0:
            contacts_list.append({
                "start": residue,
                "end": residue,
                "type": ", ".join(binders),  # Ensuring unique binders
                "description": f"Inferred from {', '.join(pdbs)}.",
                "reference": "https://www.ebi.ac.uk/pdbe/pisa/"
            })

    return contacts_list

In [19]:
for abbreviated_name, prot_seq in entry_uniprotID_seq.items():
    print(abbreviated_name)
    #if not os.path.exists(f'../examples/json_entries/new_chimeras_2/{abbreviated_name.upper()}.json'):
    if "a" == "a":
    # if abbreviated_name == "ACM3_HUMAN_ACM4_HUMAN_1":

        class_ = 'A & A' #always class A for now
        uniprotID,given_name,cutting_points,ref_parent_name,ref_parent_id,target_parent_name,target_parent_id,structures_pdb,mutations_vs_parents,gprot,biblio = retrieve_chimera_design_info(abbreviated_name)

        ref_parent_seq,target_parent_seq = retrieve_seq_parents(ref_parent_id,target_parent_id)

        #pharma name based on pharma name parents
        pharma_name_ref,pharma_name_target,pharma_name = retrieve_pharma_name_parents(ref_parent_name,ref_parent_id,target_parent_name,target_parent_id)
        
        species = species_uniprot(ref_parent_id,target_parent_id)
        
        family,subclass_ligand,subclass_phylo=familly_subclass_parents(ref_parent_name,ref_parent_id,target_parent_name,target_parent_id)

        #Secondary structure info
        allregions = compute_dssp_TM_regions(uniprotID,MSA,type_gpcr = "chimera",ref_id=ref_parent_id, seq_chimera = prot_seq)
        #structures
        structures, interacting_residues_list, binders_list, pdbs = retrieve_pdb_dsbonds_chimera_info(structures_pdb,uniprotID,prot_seq)
        structures = retrieve_predicted_models(structures,uniprotID)
        
        #retrieve the residues interacting with ligand/Gprot/Nb/Ab in PDB and link it to region
        #Add manually extra IC and EC contacts
        #should follow the following structure: list regrouping all dictionaries with 1 dict per contact
        #in dictionary: {"start":,"end","type","description","reference"}
        #for the EC contacts the types can be "orthosteric","allosteric","VHH EC"
        #for the IC contacts the types can be "G-protein","VHH IC"
        # manual_ICs,manual_ECs = translate_interacting_residues_IC_EC(interacting_residues_list,binders,pdbs,allregions)
        # contacts = translate_interacting_residues_IC_EC(interacting_residues_list,binders,pdbs,allregions)
        contacts =gather_interacting_residues(interacting_residues_list,binders_list,pdbs)
        disulfide_bonds = disulfide_bonds_chimera(ref_parent_id,ref_parent_seq,prot_seq,allregions)

        #chimeric info
        chimeras_ref_involved = retrieve_involvement_natural_chimeric_design(ref_parent_id,allregions)
        chimeras_target_involved = retrieve_involvement_natural_chimeric_design(target_parent_id,allregions)  
        chimeras = chimeras_ref_involved + chimeras_target_involved
        chimeras_no_duplicates = {}
        for d in chimeras:
            chimeras_no_duplicates[d['name']] = d
        chimeras_no_duplicates = list(chimeras_no_duplicates.values())

        confo_biosensor = []

        #chimeric info specfic design being studied
        for d in chimeras_no_duplicates:
            if d.get("name") == abbreviated_name:
                info_specific_chimera = d.copy()
        info_specific_chimera["mutations vs parents"] = mutations_vs_parents
        info_specific_chimera["ref parent seq"] = ref_parent_seq
        info_specific_chimera["target parent seq"] = target_parent_seq
        info_specific_chimera["coloring"] =sections_coloring_chimera(info_specific_chimera,allregions)

        #microswitches based on microswitches/motifs in parents
        microswitches, BW_numbering = motifs_microswitches_BWnumbering_chimera(allregions,ref_parent_id,target_parent_id,info_specific_chimera, MSA)
        
        info = {}
        #standard needed in json chimera for DB
        info["chimera"] = True
        info["ancestors"] = [ref_parent_name,target_parent_name]

        info["info specific chimera"]=info_specific_chimera

        #Abbreviated name
        info["Abbreviated name"] = [{"value": abbreviated_name.upper(), "reference": "GPCRchimeraDB"}]

        #pharma name
        info["Pharmacological name"] = [{"value": pharma_name, "reference": "GPCRdb"}]

        #Name
        info["Name(s)"] = [{"value": given_name, "reference": "GPCRchimeraDB"}]

        #Uniprot ID
        info["Uniprot ID"] = [{"value": uniprotID, "reference": "GPCRchimeraDB"}]

        #Species
        info["Organism"] =  [{"value":species, "reference": "UniProt"}]

        #Class
        info["Class"] = [{"value":class_, "reference": "GPCRdb"}]

        #Family
        info["Family"] = [{"value": family, "reference": "GPCRdb"}]

        #Subclass
        #Phylogenetically based & Ligand based
        info["Subclass"] = {"Phylogenetically based": [{"value": subclass_phylo, "reference": "10.1124/mol.63.6.1256"}],
                            "Ligand based": [{"value":subclass_ligand, "reference": "GPCRdb"}]}

        #endogenous ligand
        info["Endogenous ligand"]= []
        
        #Gport and Barr coupling data
        if len(gprot)>0:
            info["G-protein coupling"]= [{"value":gprot, "reference": biblio}]
        else:
            info["G-protein coupling"]= []
        info["Beta-arrestin coupling"] = []

        #Structures
        info["Structures"] = structures

        #Info related to chimeric design
        info["Conformational biosensor"] = confo_biosensor
        info["Involvement in chimeric design"] = chimeras_no_duplicates
        info["Cutting point values"] = write_cutting_point_region(cutting_points,ref_parent_name,target_parent_name,biblio)

        features = {}
        features['Microswitches'] = microswitches
        features['PTMs'] = []
        features['Disulfide bonds'] = disulfide_bonds
        features['Mutagenesis'] = []
        features['Pharmacological mutagenesis'] = []
        features["Contacts"] = contacts
        info["Features"] = features

        #Sequence
        info["Sequence"] = [{"value":prot_seq, "reference": biblio}]

        #Secondary structure info
        info["Limits regions"] = allregions

        #BW numbering
        info["BWnumbering"] = [{"value":BW_numbering, "reference": "MSA"}] 

        #Gather info that could be useful for chimeric design: for chimera put just yes chimera so that filter can be used on DB?
        known_info = [{"value": "Yes chimera"}]
        info["Known info chimeric design"] = known_info
        info["entryType"] = "chimera"

        json.dump(info, open(f'../examples/json_entries/new_chimeras_2/{abbreviated_name.upper()}.json', 'w'), indent=2)

OPSD_BOVIN_ACM3_HUMAN_1
OPRM_MOUSE_OPRK_HUMAN_1
NTR1_HUMAN_OPRK_HUMAN_1
SSR2_HUMAN_OPRK_HUMAN_1
HRH2_HUMAN_OPRK_HUMAN_1
ADA1A_HUMAN_OPRK_HUMAN_1
CXCR3_HUMAN_OPRK_HUMAN_1
ADA1A_HUMAN_OPRK_HUMAN_2
